---
format: 
  html:
    toc: true
    number-sections: true
---

# Trim Additional Data

**Do not Re-run!!**

To enhance the efficiency of data analysis, this section is used to pre-process other large datasets and store them as clean, reduced datasets created for analysis purposes. Additional data gathered and cleaned include Department of Health and Mental Hygiene (DOHMH) New York City Restaurant Inspection Results and additional 311 request records.

*This section should not be re-run due to its sole purpose for the creation of more manageable datasets.*

source: 
- DOHMH New York City Restaurant Inspection Results, NYC Open Data [https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j/about_data]
- 311 Service Requests from 2010 to Present, NYC Open Data [https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/about_data]

## Set Up

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

from datetime import datetime

## DOHMH New York City Restaurant Inspection Results

First, we loaded the New York City Restaurant Results dataset and extracted all unique violation types. While the violation types are highly descriptive, we note that terms like "mice," "roaches," and "vermin" are indicative of a restaurant's high vulnerability to trash mismanagement. Using a complete list of these indicative words, we filtered the dataset to include only trash-related conditions and stored it for further use.

In [2]:
res_inspection = pd.read_csv("data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv")

In [3]:
violation_types = res_inspection['VIOLATION DESCRIPTION'].unique()

print(violation_types)

[nan 'Thawing procedure improper.'
 'Hot TCS food item not held at or above 140 °F.'
 'Hot food item not held at or above 140º F.'
 "Evidence of mice or live mice in establishment's food or non-food areas."
 'Hand wash sign not posted'
 "Live roaches in facility's food or non-food area."
 'Current letter grade or Grade Pending card not posted'
 'Current letter grade sign not posted.'
 'Food contact surface not properly maintained.'
 'Wiping cloths soiled or not stored in sanitizing solution.'
 'Ashtrays in smoke-free area' 'Thawing procedures improper.'
 'Wiping cloths not stored clean and dry, or in a sanitizing solution, between uses.'
 'Permit not conspicuously displayed.'
 'Equipment used for ROP not approved by the Department'
 'Wash hands sign not posted near or above hand washing sink.'
 "Live roaches present in facility's food and/or non-food areas."
 "Evidence of rats or live rats in establishment's food or non-food areas."
 'MISBRANDED AND LABELING'
 'Workplace SFAA policy no

In [4]:
violation_list = ["sewage",
                 "mice",
                  "roaches",
                 "pest",
                 "rodents",
                 "harborage",
                 "vermin",
                 "filth",
                 "flies",
                 "garbage",
                 "receptacle",
                 "organics",
                 "nuisance",
                 "waste",
                 "back-flow"]

In [5]:
# Create regex pattern from complaint_list
violation_pattern = "|".join(violation_list)  # Combine terms into a regex pattern

# Filter for specific complaint types in 'VIOLATION DESCRIPTION'
res_inspection_filtered = res_inspection[
    res_inspection['VIOLATION DESCRIPTION'].str.contains(violation_pattern, case=False, na=False)
]

In [6]:
# Store trimmed data
res_inspection_filtered.to_csv("data/sanitation_related_restaurant_inspections.csv", index=False)

## Other 311 Requests

Additionally, we extracted complaint records for "derelict vehicles" and "graffiti" from the 311 request parquet. Like trash-related requests, these complaints are also handled by DSNY and reflect poorly managed locations prone to trash dumping. We filtered and stored the records associated with these two complaints separately for further use.

In [7]:
# Load the Master Parquet file
requests = pd.read_parquet('data/311_Service_Requests.parquet')

In [8]:
filtered_requests = requests[
    (requests['Agency'] == 'DSNY')]

In [9]:
# Create a column of the complaint creation date
filtered_requests = filtered_requests.rename(columns={'Created Date': 'Created Date Time'})
filtered_requests = filtered_requests.rename(columns={'Closed Date': 'Closed Date Time'})

filtered_requests['Created Date'] = pd.to_datetime(filtered_requests['Created Date Time'], format='%m/%d/%Y %I:%M:%S %p').dt.date

# Drop redundant columns
filtered_requests = filtered_requests.drop(columns=['Created Date Time', 'Closed Date Time'])

start_date = datetime.strptime('2022-01-01', '%Y-%m-%d').date()
end_date = datetime.strptime('2023-12-31', '%Y-%m-%d').date()

### Derelict Vehicles

In [10]:
derelict_vehicles = filtered_requests[
    (filtered_requests['Complaint Type'] == "Derelict Vehicles") &
    (filtered_requests['Created Date'] >= start_date) & 
    (filtered_requests['Created Date'] <= end_date)
]

In [11]:
# Store trimmed data
derelict_vehicles.to_parquet('data/derelict_vehicles.parquet', engine='pyarrow', compression='snappy', index=False)

### Graffiti

In [12]:
graffiti = filtered_requests[
    (filtered_requests['Complaint Type'] == "Graffiti") &
    (filtered_requests['Created Date'] >= start_date) & 
    (filtered_requests['Created Date'] <= end_date)
]

In [13]:
# Store trimmed data
graffiti.to_parquet('data/graffiti.parquet', engine='pyarrow', compression='snappy', index=False)

### Vacant Lot

We considered adding vacant lot as a factor, but there is no data reported under this category for the specified years.